# Квазиэкспериментальные исследовательские дизайны
1. В некоторых случаях проведение эксперимента невозможно в силу этических и ресурсных ограничений.
2. На помощь приходят естественные экперименты и квазиэкспериментальные исследовательские дизайны.
3. Мы рассмотрим несколько вариантов данных дизайнов в рамках текущего занятия. 

In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from plotnine import *

## Модель разность разностей
1. Идея - рассмотреть две группы, одна из конторых получает экспериментальное воздействие, а другая его не получает.
2. По каждой из групп есть два наблюдения, до получения воздействия и после.
3. Кард и Крюгер - известная статья, где был предложен данный дизайн исследования.
4. Повышение МРОТ в Нью-Джерси при отсутствии повышения МРОТ в Пенсивальнии. 
5. Сэмпл - рестораны быстрого обслуживания на границе двух штатов (почему?).
6. Две волны опроса, до повышения МРОТ в Нью-Джерси и после повышения МРОТ в Нью-Джерси. 

In [3]:
#загрузим данные Карда и Крюгера
colnames = ['SHEET', 'CHAIN', 'CO_OWNED', 'STATE', 'SOUTHJ', 'CENTRALJ', 'NORTHJ', 'PA1', 'PA2', 'SHORE', 'NCALLS', 'EMPFT',
 'EMPPT', 'NMGRS', 'WAGE_ST', 'INCTIME', 'FIRSTINC', 'BONUS', 'PCTAFF', 'MEALS', 'OPEN', 'HRSOPEN', 'PSODA', 'PFRY',
 'PENTREE', 'NREGS', 'NREGS11', 'TYPE2', 'STATUS2', 'DATE2', 'NCALLS2', 'EMPFT2', 'EMPPT2', 'NMGRS2', 'WAGE_ST2', 
 'INCTIME2', 'FIRSTIN2', 'SPECIAL2', 'MEALS2', 'OPEN2R', 'HRSOPEN2', 'PSODA2', 'PFRY2', 'PENTREE2', 'NREGS2',
 'NREGS112']     
data = pd.read_fwf("public.dat", header = None, names = colnames)
data

,SHEET,CHAIN,CO_OWNED,STATE,SOUTHJ,CENTRALJ,NORTHJ,PA1,PA2,SHORE,...,FIRSTIN2,SPECIAL2,MEALS2,OPEN2R,HRSOPEN2,PSODA2,PFRY2,PENTREE2,NREGS2,NREGS112
0,46,1,0,0,0,0,0,1,0,0,...,0.08,1,2,6.50,16.50,1.03,.,0.94,4,4
1,49,2,0,0,0,0,0,1,0,0,...,0.05,0,2,10.00,13.00,1.01,0.89,2.35,4,4
2,506,2,1,0,0,0,0,1,0,0,...,0.25,.,1,11.00,11.00,0.95,0.74,2.33,4,3
3,56,4,1,0,0,0,0,1,0,0,...,0.15,0,2,10.00,12.00,0.92,0.79,0.87,2,2
4,61,4,1,0,0,0,0,1,0,0,...,0.15,0,2,10.00,12.00,1.01,0.84,0.95,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,423,2,1,1,0,0,1,0,0,0,...,0.50,0,1,11.00,11.00,1.05,0.84,2.32,3,2
406,424,2,1,1,0,0,1,0,0,0,...,0.50,0,1,11.00,14.00,1.05,0.94,2.32,5,3
407,426,3,1,1,0,0,1,0,0,0,...,0.25,1,2,6.00,18.00,1.11,1.05,1.05,6,5
408,427,4,0,1,0,0,1,0,0,0,...,.,1,2,10.50,12.50,1.11,1.09,2.07,2,2


Единица анализа в массиве - ресторан, результаты для двух волн представлены суффиксами 0 и 1. Т.к. Wage_0 обозначает зарплату в данном ресторане, зафиксированную первой волной опроса, а Wage_1 -  зарплата в том же ресторане, зафиксированная во вторую волну опроса. 
## Задача 1
Переформатируйте (сделайте решейпинг) данного массива, чтобы в качестве единиц анализа был ресторан - период (0 - до введения МРОТ в Нью-Джерси, 1 - после введения МРОТ в Нью-Джерси). Массив должен включать следующие колонки (названия могут быть немного другими в зависимости от того, как вы обработаете суффиксы в функциях решейпинга):
State, Store ID, Wage, Employees, Entree, Tbraise. Перед решейпингом замените "." на np.nan с помощью метода replace из библиотеки pandas. 

In [7]:
#выберем нужные переменные
data_estimation = pd.DataFrame({'State': data['STATE'],
                                'Store ID': data['SHEET'],
                                'Wage_0': data['WAGE_ST'], #this is wage pre-reform
                                'Wage_1': data['WAGE_ST2'], #wage post-reform
                                'Employees_0': data['EMPFT'], #number of employees pre-reform
                                'Employees_1': data['EMPFT2'], #number of employees post-reform
                                'Entree_0': data['PENTREE'], #price of entree (basic meal) pre-reform
                                'Entree_1': data['PENTREE2'], #price of entree (basic meal) post-reform
                                'Tbraise_0': data['INCTIME'], #time before the salary raise - pre-reform
                                'Tbraise_1': data['INCTIME2']}) #time after the salary raise - post-reform 
#заменим "." на np.nan
data_estimation.replace(".", np.nan, inplace = True)
#решейпинг
data_estimationw = pd.wide_to_long(data_estimation, stubnames = ['Wage_', 'Employees_', 'Entree_', 'Tbraise_'], i = ['State','Store ID'], j = 'Treatment_Time')
#переделаем индекс в колонки
data_estimationw.reset_index(inplace = True)
#поменяем тип данных
data_estimationw = data_estimationw.astype(float)
data_estimationw

,State,Store ID,Treatment_Time,Wage_,Employees_,Entree_,Tbraise_
0,0.0,46.0,0.0,NaN,30.0,0.52,19.0
1,0.0,46.0,1.0,4.30,3.5,0.94,26.0
2,0.0,49.0,0.0,NaN,6.5,2.35,26.0
3,0.0,49.0,1.0,4.45,0.0,2.35,13.0
4,0.0,506.0,0.0,NaN,3.0,2.33,13.0
...,...,...,...,...,...,...,...
815,1.0,426.0,1.0,5.05,5.0,1.05,19.0
816,1.0,427.0,0.0,4.75,7.0,2.03,13.0
817,1.0,427.0,1.0,5.05,0.0,2.07,NaN
818,1.0,428.0,0.0,4.62,0.0,1.06,13.0


In [29]:
#посмотрим на саммари статистики
data_estimationw.describe()

,State,Store ID,Treatment_Time,Wage_,Employees_,Entree_,Tbraise_
count,820.000000,820.000000,820.000000,779.000000,802.000000,784.000000,723.000000
mean,0.807317,246.507317,0.500000,4.805712,8.238778,1.337883,20.080913
std,0.394647,148.141276,0.500305,0.358395,8.298807,0.646135,12.139997
min,0.000000,1.000000,0.000000,4.250000,0.000000,0.410000,2.000000
25%,1.000000,119.000000,0.000000,4.500000,2.000000,0.940000,13.000000
50%,1.000000,237.500000,0.500000,5.000000,6.000000,1.030000,19.000000
75%,1.000000,372.000000,1.000000,5.050000,12.000000,1.842500,26.000000
max,1.000000,522.000000,1.000000,6.250000,60.000000,3.950000,52.000000


In [11]:
#оценим модель для МРОТ - проверка на здравый смысл
model = sm.regression.linear_model.OLS.from_formula('Wage_ ~ Treatment_Time + State + Treatment_Time*State',
                                                    data = data_estimationw)
results = model.fit()
print(results.summary())
#действительно, МРОТ повысился

                            OLS Regression Results                            
Dep. Variable:                  Wage_   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     177.5
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           1.37e-87
Time:                        11:30:34   Log-Likelihood:                -101.80
No. Observations:                 779   AIC:                             211.6
Df Residuals:                     775   BIC:                             230.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                4.6301 

In [23]:
#теперь оценим модель для занятости 
model = sm.regression.linear_model.OLS.from_formula('Employees_ ~ Treatment_Time + State + Treatment_Time*State',
                                                    data = data_estimationw)
results = model.fit()
print(results.summary())
#заметьте, что F-статистика не значима

                            OLS Regression Results                            
Dep. Variable:             Employees_   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     2.122
Date:                Wed, 18 Dec 2024   Prob (F-statistic):             0.0959
Time:                        11:44:52   Log-Likelihood:                -2831.4
No. Observations:                 802   AIC:                             5671.
Df Residuals:                     798   BIC:                             5690.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               10.2051 

In [25]:
#посмотрим на цену базового блюда (как правило, бургер)
model = sm.regression.linear_model.OLS.from_formula('Entree_ ~ Treatment_Time + State + Treatment_Time*State',
                                                    data = data_estimationw)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                Entree_   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     3.173
Date:                Wed, 18 Dec 2024   Prob (F-statistic):             0.0237
Time:                        11:46:33   Log-Likelihood:                -764.78
No. Observations:                 784   AIC:                             1538.
Df Residuals:                     780   BIC:                             1556.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                1.2151 

In [26]:
#наконец, посмотрим на время до повышения зарплаты 
model = sm.regression.linear_model.OLS.from_formula('Tbraise_ ~ Treatment_Time + State + Treatment_Time*State',
                                                    data = data_estimationw)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               Tbraise_   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     6.056
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           0.000449
Time:                        11:47:29   Log-Likelihood:                -2821.3
No. Observations:                 723   AIC:                             5651.
Df Residuals:                     719   BIC:                             5669.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               18.8716 

## Инструментальные переменные
1. В анализе причинно-следственных связей часто можно встретить проблему spurious correlation, когда третья переменная одновременно влияет и на независимую (X), и на зависимую (Y) переменную. 
2. Например, уровень экономического благополучию регионов может одновременно влиять на распространение социальных сетей и протестную активность. 
3. Или же институациональные характеристики государства могут влиять и на экономическую ситуацию, и на возникновение в нём гражданского конфликта. 
4. Решение - найти переменную Z, такую, что она не влияет напрямую на Y, но влияет на X. 
5. Друзья Павла Дурова, количество выпавших осадков и т.д. 

## Acemoglu, Robinson and Johnson - Colonial Origins of Comparative Development
1. Исследовательский вопрос - как экономические институты (прежде всего, защита прав частной собственности) влияют на уровень экономического развития? 
2. Во-первых, возможна spurious correlation; во-вторых, возможен обратный эффект. 
3. Как изолировать каузальный эффект? 

In [31]:
#загрузим данные
data = pd.read_csv("Acemoglu_et_al.csv")
data.describe()

,catho80,muslim80,lat_abst,no_cpm80,f_brit,f_french,avexpr,sjlofr,logpgp95,logem4,baseco
count,162.000000,162.000000,162.000000,160.000000,162.000000,162.000000,121.000000,162.000000,148.000000,87.000000,64.0
mean,30.040123,25.288519,0.295550,32.707250,0.308642,0.148148,7.066491,0.469136,8.302509,4.595984,1.0
std,35.610717,36.880691,0.190379,32.187160,0.463365,0.356348,1.804287,0.500594,1.105342,1.303333,0.0
min,0.000000,0.000000,0.000000,0.100003,0.000000,0.000000,1.636364,0.000000,6.109248,0.936093,1.0
25%,0.700000,0.000000,0.144444,4.349998,0.000000,0.000000,5.886363,0.000000,7.376192,4.224609,1.0
50%,10.600000,2.050000,0.266667,20.999999,0.000000,0.000000,7.045454,0.000000,8.265764,4.442651,1.0
75%,55.174999,42.250000,0.446944,51.825000,1.000000,0.000000,8.272727,1.000000,9.216229,5.610119,1.0
max,97.300003,99.800003,0.722222,100.000000,1.000000,1.000000,10.000000,1.000000,10.288750,7.986165,1.0


In [32]:
#функция для оценки модели с инструментальной переменной
def ivregress(DV, endogenous_variable, instrument, exog, data):
    data['Intercept'] = 1
    data_estimation_1 = data[['Intercept'] + [instrument] + exog]
    model_1 = sm.regression.linear_model.OLS(endog = data[endogenous_variable], exog = data_estimation_1, missing = "drop")
    results_1 = model_1.fit()
    #obtain predictions
    data[endogenous_variable + "iv"] = np.dot(data_estimation_1, results_1.params)
    #create estimation dataset for the second stage
    data_estimation_2 = data[['Intercept'] + [endogenous_variable + "iv"] + exog]
    model_2 = sm.regression.linear_model.OLS(endog = data[DV], exog = data_estimation_2, missing = "drop")
    results_2 = model_2.fit()
    return (results_2, results_1, data_estimation_2)

In [33]:
data_estimation = data.loc[data.baseco==1].copy()
results = ivregress('logpgp95', 'avexpr', 'logem4', 
                ['lat_abst', 'f_french', 'sjlofr', 'catho80', 'muslim80', 'no_cpm80'], data_estimation)
#результаты оценки модели
print(results[0].summary())

                            OLS Regression Results                            
Dep. Variable:               logpgp95   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.540
Method:                 Least Squares   F-statistic:                     11.57
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           5.46e-09
Time:                        12:04:17   Log-Likelihood:                -64.393
No. Observations:                  64   AIC:                             144.8
Df Residuals:                      56   BIC:                             162.1
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4472      1.648      0.271      0.7

In [34]:
#проверить качество инструмента
print(results[1].summary())

                            OLS Regression Results                            
Dep. Variable:                 avexpr   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     4.671
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           0.000355
Time:                        12:07:19   Log-Likelihood:                -100.19
No. Observations:                  64   AIC:                             216.4
Df Residuals:                      56   BIC:                             233.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.3273      1.685      4.943      0.0

In [39]:
#регрессия без институтов
model = sm.regression.linear_model.OLS.from_formula('logpgp95 ~ lat_abst + f_french + sjlofr + catho80 + muslim80 + no_cpm80',
                                                    data = data_estimation).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               logpgp95   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     6.824
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           1.72e-05
Time:                        12:11:55   Log-Likelihood:                -75.701
No. Observations:                  64   AIC:                             165.4
Df Residuals:                      57   BIC:                             180.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.5983      0.887      8.567      0.0

In [40]:
#регрессия с институтами
model = sm.regression.linear_model.OLS.from_formula('logpgp95 ~ avexpr + lat_abst + f_french + sjlofr + catho80 + muslim80 + no_cpm80',
                                                    data = data_estimation).fit()
print(model.summary())
#эффект институтов несколько меньший, но присутствует и здесь 

                            OLS Regression Results                            
Dep. Variable:               logpgp95   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     20.78
Date:                Wed, 18 Dec 2024   Prob (F-statistic):           1.80e-13
Time:                        12:12:47   Log-Likelihood:                -52.060
No. Observations:                  64   AIC:                             120.1
Df Residuals:                      56   BIC:                             137.4
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.8184      0.713      6.756      0.0

## Разрывная регрессия
1. В отличие от модели разность разностей, используется кросс-секционный дизайн. 
2. Требования к данным - существование порога, влияющего на получение воздействия.
3. В строгом (sharp) дизайне порог предполагает $\mathbb{P}[Assignment = 1| X \leq threshold] = 0$,  $\mathbb{P}[Assignment = 1| X > threshold] = 1$.
4. В нестрогом (fuzzy) дизайне порог предполагает, что $\mathbb{P}[Assignment = 1| X \leq threshold] <  \mathbb{P}[Assignment = 1| X > threshold]$. 

## Разрывная регрессия - Пример 1
1. Campbell - эффект получения школьных наград на дальнейшие успехи в образовании и карьере. 
2. Порог - балл в тесте, гарантирующий получение награды.
3. Идентификационное предположение - разницу между теми, кто немного недобрал до балла и теми, кто набрал чуть больше проходного балла, можно считать случайной.

## Разрывная регрессия - Пример 2
1. Dell, Drug Cartels - эффект партийной аффилиации мэров на drug-related violence. 
2. В 2006 году Кальдерон выиграл выборы и стал Президентом Мексики, одновременно начав кампанию крэкдаунов против наркокартелей.
3. Реализация кампании на местах зависела от глав муниципалитетов - там, где локальная власть контролировалась партией Кальдерона, кампания велась активно, в других местах она велась пассивнее. 
4. Идея - оценить эффект кампании, сравнив уровень насилия, связанного с наркоторговлей, в муниципалитетах с мэрами из партии Кальдерона и муниципалитетах с другими мэрами.
5. Берутся муниципалитеты, где мэры "от Кальдерона" либо выиграли с минимальным перевесом, либо проиграли с минимальным перевесом. 

## Разрывная регрессия - Пример 3
1. Ferwerda and Miller - эффекты иностранного управления в военных конфликтах.
2. Кейс - разделённая Франция времён Второй мировой войны. Север Франции управлялся напрямую нацистской администрацией, а Юг Франции - марионеточным правительством под руководством маршала Филиппа Петена (Вишистская Франция - по названию города Виши, где находилось правительство). 
3. Квази-эксперимент - две части Франции были разделены по линии железной дороги.
4. Сравнение активности движения сопротивления нацистам в communes, расположенных вблизи железнодорожного полотна. 

## Метод синтетического контроля
1. Что, если воздействие применялось только к одному объекту?
2. Метод синтетического контроля - взвешивание контрольных наблюдений таким образом, чтобы получившаяся линия тренда наилучшим образом аппроксимировала наблюдений для объекта, подвергшагося воздействию.
3. Неободимо достаточное количество наблюдений в рамках временного ряда по каждому объекту изучения. 